In [ ]:
# -*- coding: utf-8 -*-
"""Speech Processing Script with GUI for Equalizer and Plotting"""

import numpy as np
import scipy.signal as sp_signal
import matplotlib.pyplot as plt
import soundfile as sf
from pydub import AudioSegment
import tkinter as tk
from tkinter import filedialog, messagebox

# Define paths to the input and output files
input_file = 'racing.wav'  # Replace with your local audio file name
output_file = 'equalized_speech.wav'

# Convert .m4a to .wav if needed using pydub
if input_file.endswith('.m4a'):
    audio = AudioSegment.from_file(input_file, format='m4a')
    audio.export('converted_audio.wav', format='wav')
    input_file = 'converted_audio.wav'
    print(f'Converted .m4a to .wav: {input_file}')

# Load the converted .wav audio data
original_signal, sample_rate = sf.read(input_file)

# Convert to mono if the audio is stereo
if len(original_signal.shape) > 1:
    original_signal = np.mean(original_signal, axis=1)

# Define frequency bands for the equalizer (e.g., low, mid, and high)
freq_bands = {
    'low': (300, 600),
    'mid': (1000, 2000),
    'high': (2000, 3400)
}

# Function to convert dB gains to linear scale
def dB_to_linear(dB):
    return 10**(dB / 20)

# Filter design helper function
def bandpass_filter(data, band, sample_rate, gain):
    low, high = band
    sos = sp_signal.butter(4, [low / (sample_rate / 2), high / (sample_rate / 2)], btype='band', output='sos')
    filtered = sp_signal.sosfilt(sos, data)
    return filtered * gain

# Function to plot frequency response
def plot_frequency_response(original_signal, equalized_signal, sample_rate):
    plt.figure(figsize=(10, 6))
    
    # Original Signal Spectrogram
    plt.subplot(2, 1, 1)
    freqs, times, Sxx = sp_signal.spectrogram(original_signal, sample_rate)
    plt.pcolormesh(times, freqs, 10 * np.log10(Sxx), shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [s]')
    plt.title('Original Signal')
    plt.colorbar(label='Intensity [dB]')
    
    # Equalized Signal Spectrogram
    plt.subplot(2, 1, 2)
    freqs, times, Sxx = sp_signal.spectrogram(equalized_signal, sample_rate)
    plt.pcolormesh(times, freqs, 10 * np.log10(Sxx), shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [s]')
    plt.title('Equalized Signal')
    plt.colorbar(label='Intensity [dB]')
    
    plt.tight_layout()
    plt.show()

# Function to apply equalization based on current gain levels
def apply_equalization():
    gain = {
        'low': dB_to_linear(low_gain_slider.get()),
        'mid': dB_to_linear(mid_gain_slider.get()),
        'high': dB_to_linear(high_gain_slider.get())
    }
    
    # Apply filters for each frequency band
    filtered_signal = np.zeros_like(original_signal)
    for band, (low_freq, high_freq) in freq_bands.items():
        filtered_band = bandpass_filter(original_signal, (low_freq, high_freq), sample_rate, gain[band])
        filtered_signal += filtered_band

    # Normalize the equalized signal to avoid low volume
    filtered_signal = filtered_signal / np.max(np.abs(filtered_signal))
    
    # Save the equalized audio
    sf.write(output_file, filtered_signal, sample_rate)
    messagebox.showinfo("Equalizer", f"Equalized audio saved as {output_file}")
    print(f'Equalized audio saved as {output_file}')
    
    # Plot frequency response of original and equalized signals
    plot_frequency_response(original_signal, filtered_signal, sample_rate)

# Set up GUI
root = tk.Tk()
root.title("Simple Equalizer")

# Low frequency band slider
tk.Label(root, text="Low Frequency Gain (dB)").grid(row=0, column=0, padx=10, pady=5)
low_gain_slider = tk.Scale(root, from_=-10, to=10, orient=tk.HORIZONTAL)
low_gain_slider.set(5)  # Default boost for low frequencies
low_gain_slider.grid(row=0, column=1, padx=10, pady=5)

# Mid frequency band slider
tk.Label(root, text="Mid Frequency Gain (dB)").grid(row=1, column=0, padx=10, pady=5)
mid_gain_slider = tk.Scale(root, from_=-10, to=10, orient=tk.HORIZONTAL)
mid_gain_slider.set(-3)  # Default cut for mid frequencies
mid_gain_slider.grid(row=1, column=1, padx=10, pady=5)

# High frequency band slider
tk.Label(root, text="High Frequency Gain (dB)").grid(row=2, column=0, padx=10, pady=5)
high_gain_slider = tk.Scale(root, from_=-10, to=10, orient=tk.HORIZONTAL)
high_gain_slider.set(4)  # Default boost for high frequencies
high_gain_slider.grid(row=2, column=1, padx=10, pady=5)

# Apply button
apply_button = tk.Button(root, text="Apply Equalization", command=apply_equalization)
apply_button.grid(row=3, column=0, columnspan=2, pady=10)

# Run the GUI event loop
root.mainloop()


c:\Users\mabd0\anaconda3\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
c:\Users\mabd0\anaconda3\Lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)
c:\Users\mabd0\anaconda3\Lib\site-packages\pydub\utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
c:\Users\mabd0\anaconda3\Lib\site-packages\pydub\utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
c:\Users\mabd0\anaconda3\Lib\site-packages\pydub\utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', t

FileNotFoundError: [WinError 2] The system cannot find the file specified